# Un Algoritmo Genético Simple en Python

## Algoritmos Genéticos

Los algoritmos genéticos son algoritmos de optimización que imitan el proceso de selección natural. En lugar de utilizar "trucos matemáticos", simplemente copian una lógica de la que sabemos que funciona.

## Selección Natural en Algoritmos Genéticos

Este proceso de selección natural se basa en la supervivencia del más apto: el proceso en la naturaleza que hace que los mejores individuos (animales, plantas u otros) sobrevivan. Los individuos más aptos luego se aparean entre sí, dando lugar a una nueva generación. La naturaleza también agrega un poco de aleatoriedad en forma de mutaciones al genoma.

La nueva generación es una mezcla de individuos buenos y malos, pero aquí nuevamente, los buenos sobrevivirán, luego se aparearán y luego darán lugar a una nueva generación.
El resultado es una mejora constante de generación en generación.

## Problema

Para recalcar la teoría que acaba de aprender, veamos un problema simple:
 
> Dados los dígitos del 0 al 9 y los operadores +, -, * y /, encuentre una secuencia que represente un número objetivo determinado. Los operadores se aplicarán secuencialmente de izquierda a derecha a medida que lea.

In [1]:
# import bibliotecas principales

from __future__ import print_function
import random
import math
import sys

from operator import itemgetter
from bisect import bisect_right
from pprint import pprint

Entonces, dado el número objetivo 15, la secuencia 6 + 2 * 12 / 2 - 3 sería una posible solución.

### Algoritmo Genético

1. Codifique las características del problema en un conjunto de genes (probablemente binarios)<br>
    Genere una población de individuos (cromosomas) que combine genes aleatorios
2. ¿Qué tan bueno es cada individuo (cromosoma) para resolver el problema? (asigne una "puntuación de aptitud física")
3. Seleccione 2 individuos (cromosomas) de la población (cambio de selección proporcional a la "puntuación de aptitud")
4. Realice el cruce de genes después de una ubicación aleatoria de acuerdo con la "tasa de cruce"
5. Invierta los bits de cromosomas usando la "tasa de mutación"
6. Repita los pasos 2 a 5 hasta que el tamaño de población que desee

>TODO: Las siguientes 4 constantes pueden ser anuladas por los parámetros de llamada al script (sé que es de mal estilo, pero es un sustituto antes de convertir este script para usar clases)

In [2]:
MUTATION_RATE   = 0.03
CROSSOVER_RATE  = 0.7
CHROMO_LEN      = 10     # genes
POP_SIZE        = 100

OPERATORS       = ['+', '-', '*', '/']
NUMBERS         = list(range(10))
GENES           = NUMBERS + OPERATORS + [None]   # todos los genes posibles

GENE_BIT_MAPPING = { gene: i for i, gene in enumerate(GENES) }
BIT_GENE_MAPPING = { i: gene for gene, i in GENE_BIT_MAPPING.items() }

número de bits que se utilizan para representar cada gen. Necesitamos saber esto cuando realizamos la mutación, por lo que generamos una máscara de bits con el número correcto de bits.

In [3]:
GENE_BIT_COUNT = int(math.floor(math.log(len(GENES), 2)) + 1)

### Métodos Auxiliares

In [4]:
def chromo_to_expr(chromo):
    '''
    chromo: lista de ints que se pueden asignar a genes
    '''
    return ''.join([
        str(BIT_GENE_MAPPING[g])
        for g in chromo
        if BIT_GENE_MAPPING[g] is not None
    ])


def eval_expression(expr):
    '''
    expr: secuencia de genes definida por GENES. Devuelve la evaluación de la
    expresión matemática dada. Ninguno de los genes se ignora en el cromosoma, 
    pero todo lo demás se concatena en una cadena y luego se evalúa utilizando 
    el método 'eval' de python. 
    Los errores devolverán None; de lo contrario, se devolverá el resultado.
    '''
    # elimina valores None y luego concatena todo como una cadena
    try:
        return eval(expr)
    except (ZeroDivisionError, SyntaxError):
        return None

    
def create_chromosome():
    chromo = [
        random.choice(list(GENE_BIT_MAPPING.values()))
        for _ in range(CHROMO_LEN)
    ]

    return chromo

### Evaluamos la Adaptavilidad

In [5]:
def fitness_score(chromo, target_sum):
    chromo_eval = eval_expression(chromo_to_expr(chromo))

    # cromosoma inútil?
    if chromo_eval is None: 
        return 0

    # coincide exacto?
    if chromo_eval == target_sum:
        return float('inf')

    try:
        return 1. / abs(target_sum - chromo_eval)
    except OverflowError:
        return 0 # target_sum - chromo_eval puede ser extremadamente grande a veces

### Cruce

In [6]:
def crossover(chromo_a, chromo_b):
    '''
    cruce a nivel de gen, no a nivel de bits dentro de los genes.
    Se supone que ambos cromosomas tienen la misma longitud.
    Retorna (new_chromo_a, new_chromo_b) al realizar el cruce.
    '''
    # elige un punto de cruce aleatorio
    cross_point = int(random.random()) * len(chromo_a)
    new_chromo_a = chromo_a[:cross_point] + chromo_b[cross_point:]
    new_chromo_b = chromo_b[:cross_point] + chromo_a[cross_point:]
    
    return (new_chromo_a, new_chromo_b)

### Mutación

In [7]:
def mutate(chromo, mutation_rate):
    '''
    Retorna, para el cromosoma dado, un nuevo mutado
    '''
    mutated_chromo = []

    for gene in chromo:
        while True:
            # cree la máscara de bits xor para realizar una mutación como '010001' 
            # donde 1 significa que un bit en el gen se mutará (invertirá)
            mutation_bits = sum(
                2 ** x
                for x in range(GENE_BIT_COUNT)
                if random.random() >= mutation_rate
            )
            mutated_gene = gene ^ mutation_bits
            
            # la mutación debe producir un gen válido
            
            if mutated_gene in BIT_GENE_MAPPING.keys():
                break
        
        mutated_chromo.append(mutated_gene)

    return mutated_chromo

### Algoritmo Evolutivo

In [8]:
def breed_generation(chromo_scores):
    '''
    Genere una nueva población donde los cromosomas con los puntajes de 
    aptitud más altos sean más propensos a ser elegidos para la reproducción 
    (donde pueden ocurrir cruces y mutaciones). La selección de la ruleta 
    se utiliza para realizar la selección. El nuevo tamaño de la población 
    será el mismo que el anterior, redondeado a la cantidad par más cercana.
    
    chromo_scores: lista de tuplas (chromo, score) donde score debe ser +ve
    '''
    next_gen_chromos = []
    min_score = min(c[1] for c in chromo_scores)
    max_score = max(c[1] for c in chromo_scores)
    total_score = sum(c[1] for c in chromo_scores)

    # normalizar y distribuir puntuaciones entre [0, 1) 
    # cuanto mayor sea el intervalo entre la entrada anterior 
    # y la entrada actual, mayor será la posibilidad de reproducir
    distributed_chromo_scores = []
    cumulative_norm_score = 0

    for chromo_tuple in chromo_scores:
        # si se tiene una puntuación de 0, se da a todos una nueva oportunidad
        if total_score != 0:
            norm_score = float(chromo_tuple[1]) / total_score
        else:
            norm_score = 1. / len(chromo_scores)

        distributed_chromo_scores.append(norm_score + cumulative_norm_score)
        cumulative_norm_score += norm_score

    def roulette_chromo():
        r = random.random()
        chromo_index = bisect_right(distributed_chromo_scores, r)
        
        # Existe una pequeña posibilidad debido al redondeo de punto flotante 
        # al crear distributed_chromo_scores de que el último no sea 1,00, por 
        # lo que random() podría ser más alto. Imprima una advertencia, ya que 
        # esto también podría ser una señal de un error al hacer score de chromo.
        if chromo_index >= len(chromo_scores):
            print('WARNING, intente seleccionar un cromosoma con fitness'
                  + 'relativo <= %s pero el mas alto es %s, usando el mayor de todos modos...' %
                  (r, distributed_chromo_scores[-1]), file=sys.stderr)
        return chromo_scores[min(chromo_index, len(chromo_scores) - 1)][0]

    # cruce y mutación de dos cromosomas desde rueda de ruleta
    for i in range(len(chromo_scores) // 2):
        chromo_a, chromo_b = crossover(roulette_chromo(), roulette_chromo())
        next_gen_chromos.append(mutate(chromo_a, MUTATION_RATE))
        next_gen_chromos.append(mutate(chromo_b, MUTATION_RATE))

    return next_gen_chromos

### Main

In [10]:
# TODO: Sé que es una mala práctica hacer esto, pero es un sustituto antes de 
# convertir este script en clases.

target_val = 15

chromos = [ create_chromosome() for i in range(POP_SIZE) ]
generation = 1

while True:
    chromo_scores = [
        (chromo, fitness_score(chromo, target_val))
        for chromo in chromos
    ]

    print('Gen {0:<5}, puntaje mas alto {1:<8}'.format(
        generation, max(c[1] for c in chromo_scores)))

    # imprime un par de cromosomas cada n generaciones para ver cómo son
    if generation % 100 == 0:
        pprint([chromo_to_expr(chromo) for chromo in chromos[:5]])

    perfect_chromos = [
        chromo
        for chromo, score in chromo_scores 
        if score == float('inf')
    ]

    if len(perfect_chromos) >= 1:
        break

    chromos = breed_generation(chromo_scores)
    generation += 1

print('Encontré %s una expresión perfecta para "%s" en la %s generación de tamaño %s' %
      (len(perfect_chromos), target_val, generation, POP_SIZE))

print('Soluciones:')
print('\n'.join([chromo_to_expr(chromo) for chromo in perfect_chromos]))

Gen 1    , puntaje mas alto 0.2903225806451614
Gen 2    , puntaje mas alto 0.060021436227224015
Gen 3    , puntaje mas alto 1.7777777777777777
Gen 4    , puntaje mas alto 2.9999999999999787
Gen 5    , puntaje mas alto 1.7777777777777777
Gen 6    , puntaje mas alto 0.7499999999999987
Gen 7    , puntaje mas alto 1.7777777777777777
Gen 8    , puntaje mas alto 0.8405797101449272
Gen 9    , puntaje mas alto 3.5121951219512293
Gen 10   , puntaje mas alto 0.8405797101449272
Gen 11   , puntaje mas alto 4.390243902439023
Gen 12   , puntaje mas alto 0.8923076923076931
Gen 13   , puntaje mas alto 7.047619047619081
Gen 14   , puntaje mas alto 0.1708185053380783
Gen 15   , puntaje mas alto 13.818181818181765
Gen 16   , puntaje mas alto 0.16666666666666666
Gen 17   , puntaje mas alto 180.00000000002942
Gen 18   , puntaje mas alto 0.16422287390029328
Gen 19   , puntaje mas alto 13.818181818181765
Gen 20   , puntaje mas alto 1.1200000000000006
Gen 21   , puntaje mas alto 13.818181818181765
Gen 22   , 

## Los pasos del Algoritmo Genético:

1. ¿Cómo codificar los datos para el algoritmo genético?
2. ¿Cómo evaluar la solución del algoritmo genético?
3. ¿Cómo codificar el apareamiento (Cross-Over) para el algoritmo genético?
4. ¿Cómo codificar mutaciones para el algoritmo genético?
5. ¿Cómo definir la Selección para el Algoritmo Genético?
7. ¿Cómo definir iteraciones y paradas para el algoritmo genético?